# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

csv = "city_weather.csv"
city_weather = pd.read_csv(csv)
city_weather

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Wind Speed,Max Temp
0,0,rikitea,97,PF,1619394100,73,-23.1203,-134.9692,6.31,78.30
1,1,yellowknife,90,CA,1619393863,72,62.4560,-114.3525,13.80,10.00
2,2,mayo,40,US,1619394177,42,38.8876,-76.5119,10.36,64.40
3,3,hithadhoo,80,MV,1619393821,69,-0.6000,73.0833,4.34,82.09
4,4,troitskoye,100,RU,1619394431,76,52.9821,84.6757,5.73,35.37
...,...,...,...,...,...,...,...,...,...,...
576,576,harper,100,LR,1619394197,85,4.3750,-7.7169,2.26,77.85
577,577,del rio,1,US,1619394588,23,29.3627,-100.8968,16.11,95.00
578,578,rincon,75,US,1619394589,50,32.2960,-81.2354,9.22,75.20
579,579,nuuk,75,GL,1619394180,75,64.1835,-51.7216,5.75,41.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key= g_key)

locations = city_weather[["Latitude", "Longitude"]]
humidity = city_weather["Humidity"]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_weather = city_weather.copy()
ideal_weather = ideal_weather[ideal_weather["Max Temp"] >= 75]
ideal_weather = ideal_weather[ideal_weather["Max Temp"] <= 90]
ideal_weather = ideal_weather[ideal_weather["Wind Speed"] <= 7]
ideal_weather = ideal_weather[ideal_weather["Cloudiness"] == 0]
ideal_weather = ideal_weather.dropna()
ideal_weather = ideal_weather.reset_index()
ideal_weather

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Wind Speed,Max Temp
0,89,89,conde,0,BR,1619394446,94,-7.2597,-34.9075,5.75,77.0
1,188,188,pitimbu,0,BR,1619394469,94,-7.4706,-34.8086,5.75,77.0
2,341,341,cabedelo,0,BR,1619394493,94,-6.9811,-34.8339,5.75,77.0
3,358,358,balotra,0,IN,1619394519,12,25.8333,72.2333,1.39,81.0
4,458,458,vera cruz,0,BR,1619394554,78,-12.6333,-41.0333,4.61,75.2
5,550,550,riyadh,0,SA,1619394122,24,24.6877,46.7219,4.23,78.8


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""

hotel_df

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Wind Speed,Max Temp,Hotel Name
0,89,89,conde,0,BR,1619394446,94,-7.2597,-34.9075,5.75,77.0,
1,188,188,pitimbu,0,BR,1619394469,94,-7.4706,-34.8086,5.75,77.0,
2,341,341,cabedelo,0,BR,1619394493,94,-6.9811,-34.8339,5.75,77.0,
3,358,358,balotra,0,IN,1619394519,12,25.8333,72.2333,1.39,81.0,
4,458,458,vera cruz,0,BR,1619394554,78,-12.6333,-41.0333,4.61,75.2,
5,550,550,riyadh,0,SA,1619394122,24,24.6877,46.7219,4.23,78.8,


In [13]:
hotels = []
x = 0


for cities in hotel_df["City"]:
    target_coordinates = str(hotel_df["Latitude"][x])+' , '+str(hotel_df["Longitude"][x])
    x = x + 1
    target_radius = 5000
    target_type = "lodging"
    params = {
        "location" : target_coordinates,
        "radius" : target_radius,
        "type" : target_type,
        "key" :  g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        response = requests.get(base_url, params = params)
        hotel_nearby = response.json()
        hotels.append(hotel_nearby['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)
        
hotel_df['Hotel Name'] = hotels
hotel_df_clean = hotel_df.dropna()
hotel_df_clean

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Wind Speed,Max Temp,Hotel Name
0,89,89,conde,0,BR,1619394446,94,-7.2597,-34.9075,5.75,77.0,Pousada Beija Flor
1,188,188,pitimbu,0,BR,1619394469,94,-7.4706,-34.8086,5.75,77.0,Reserva do Abiaí
2,341,341,cabedelo,0,BR,1619394493,94,-6.9811,-34.8339,5.75,77.0,Hotel Pigalle
3,358,358,balotra,0,IN,1619394519,12,25.8333,72.2333,1.39,81.0,Hotel City Square
5,550,550,riyadh,0,SA,1619394122,24,24.6877,46.7219,4.23,78.8,"InterContinental Riyadh, an IHG Hotel"


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))